In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2

# Ray casting

open3d coordinate system: right, up, out

In [32]:
import open3d as o3d
from o3d_utils import ray_geometries, visualize, rays_tensor

Setup the scene

In [3]:
meshes = []
scene = o3d.t.geometry.RaycastingScene()

# width, height, depth
box_dims = np.array([[1.0, 3.0, 1.0], [1.0, 3.0, 1.0], [3.0, 3.0, 1.0]])
box_positions = np.array([[2.0, 0, 0], [0, 0.0, -1.5], [0.5, 0.0, 1.5]])

for i in range(box_dims.shape[0]):
    box_mesh = o3d.geometry.TriangleMesh.create_box(box_dims[i, 0], box_dims[i, 1], box_dims[i, 2])
    box_mesh.translate(box_positions[i])
    meshes.append(box_mesh)
    box_tris = o3d.t.geometry.TriangleMesh.from_legacy(box_mesh)
    scene.add_triangles(box_tris)

In [4]:
# Define ray origins and directions
ray_origins = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])  # All originating from the origin
ray_directions = np.array([[1, 1, 1], [-1, 1, 0], [0, 1, 1]])  # Different directions

rays_vis = ray_geometries(ray_origins, ray_directions)

In [5]:
geometries = meshes + rays_vis
visualize(geometries)

In [ ]:
rays = rays_tensor(ray_origins, ray_directions)
result = scene.cast_rays(rays)
result

In [7]:
from o3d_utils import reflect

In [8]:
num_reflections = 1
reflect_origins = []
reflect_directions = []

for i in range(len(result['t_hit'])):
    if result['t_hit'][i] < np.inf:
        reflect_direction = reflect(ray_directions[i], result['primitive_normals'][i].numpy())
        reflect_origin = ray_origins[i] + result['t_hit'][i].numpy() * ray_directions[i]
        reflect_origins.append(reflect_origin)
        reflect_directions.append(reflect_direction)

reflected_rays_vis = ray_geometries(reflect_origins, reflect_directions)

In [9]:
geometries = meshes + rays_vis + reflected_rays_vis
visualize(geometries)

reflection formula: r = d - 2 * dot(d, n) * n
- r: reflected direction
- d: incident direction
- n: normal direction

NLOS determination
1. setup scene geometry
2. from satellite az/els, calculate ray directions from receiver (origin) to satellites
3. cast rays and check for intersection

Direct DOP prediction?
- continuous equivalent, like SDF
- "volumetric" DOP - compute from sky visibility shape

Multipath prediction
1. setup scene geometry
2. assume approximate receiver position
3. from satellite az/els, calculate nominal ray directions from satellites to receiver
4. for each nominal ray, form ray bundle of parallel rays
5. cast ray bundles and compute reflections

In [24]:
from o3d_utils import setup_scene

In [28]:
def azel2los(az, el):
    x = np.cos(az) * np.sin(el)
    y = np.sin(az) * np.sin(el)
    z = np.cos(el)
    return np.array([x, y, z])

def azel2los_vec(azels):
    x = np.cos(azels[:,0]) * np.sin(azels[:,1])
    y = np.sin(azels[:,0]) * np.sin(azels[:,1])
    z = np.cos(azels[:,1])
    return np.stack((x,y,z)).T

def enu2o3d(enu):
    # ENU: right, in, up
    # o3d: right, up, out
    o3d_pts = enu[:,[0,2,1]]
    o3d_pts[:,2] *= -1
    return o3d_pts

In [29]:
# width, height, depth
box_dims = np.array([[10.0, 30.0, 10.0], [15.0, 20.0, 10.0], [30.0, 50.0, 12.0]])
box_positions = np.array([[20.0, 0, -10.0], [-20, 0.0, 0.0], [20, 0.0, 15.0]])

scene, meshes = setup_scene(box_dims, box_positions)

In [33]:
azels = np.deg2rad(np.array([[10, 20], [40, 50], [100, 30], [240, 45]]))
nominal_directions = enu2o3d(azel2los_vec(azels))

SOURCE_DIST = 100

ray_origins = SOURCE_DIST * nominal_directions
ray_directions = -ray_origins
ray_tensor = rays_tensor(ray_origins, ray_directions)
ray_geom = ray_geometries(ray_origins, ray_directions)

In [35]:
visualize(meshes + ray_geom)

In [36]:
def ray_bundle(ray_origin, ray_direction):
    # Sample random points in ball near ray origin
    # (more proper: sample points (regularly) in disk in ray direction normal plane)
    NUM_RAYS = 5
    RADIUS = 1.0

    origins = ray_origin + RADIUS * np.random.uniform(-1.0, 1.0, (NUM_RAYS, 3))

# Satellite forecasting

Almanac is valid for predicting DOP up to 2 weeks - 90 days

https://receiverhelp.trimble.com/alloy-gnss/en-us/almanacs.html?tocpath=Receiver%20Web%20Interface%7CSatellites%20menu%7C_____6
- don't see the download link?

Ephemeris is valid 2 hours before and after

In [6]:
import gnss_lib_py as glp

In [11]:
from datetime import datetime, timezone
# Send time at which SV states are needed in GPS millis
start_time = datetime(year=2024,
                       month=4,
                       day=29,
                       hour=22,
                       minute=30,
                       second=0)
start_time = start_time.replace(tzinfo=timezone.utc)
start_gps_millis = glp.datetime_to_gps_millis(start_time)

In [ ]:
sp3_path = glp.load_ephemeris(file_type="sp3",
                              gps_millis=start_gps_millis,
                              verbose=True)

In [ ]:
rx_LLA_durand = np.reshape([37.427112, -122.1764146, 16], [3, 1])
rx_ecef_durand = np.reshape(glp.geodetic_to_ecef(rx_LLA_durand), [3, 1])